# Retrieve Round 2 NFTs by Policy ID using CardanoScan API

---

### 1. Overview
This notebook details the process of retrieving the NFTs associated with the Round 2 Teddy Bear Club Policy ID using the [CardanoScan API](https://docs.cardanoscan.io/operation/operation-get-asset-list-byaddress). The process includes securely accessing your API key, querying the API, and saving the results in a formatted JSON file. The data is then used to identify the order of minting.

The process includes:
1. Securely accessing your API key.
2. Querying the API for assets tied to the policy ID.
3. Saving the results to a formatted JSON file according to order of minting.
4. Saving the results to a CSV file mapping a subject to its order.

Before running this notebook:
- Ensure you have a valid **CardanoScan API key** set up in your environment variables.
- Verify that the address you provide is in **hexadecimal format**.

---
### 1.1 Necessary Package Installations and Open Statements

In [ ]:
#r "nuget: FSharp.Data"

In [2]:
open System
open System.IO
open System.Text.Json
open System.Net.Http
open System.Threading.Tasks
open FSharp.Data

### 2. Workflow

*Note!*<br>
To use the [CardanoScan API](https://docs.cardanoscan.io/), you must have a valid API key. The `getApiKey` function retrieves this key from your environment variables, so ensure your key is named `CARDANO_API_KEY` or update the function to match your setup. 

1. **`getApiKey`**:
   - Retrieves your API key from the environment.
   - Ensures the key is securely accessed and avoids hardcoding sensitive information.

2. **`saveJsonToFile`**:
   - Formats the JSON response into a more readable format and saves it to a file.
   - By default, it saves the output to [`collectionassets.json`](./data/collectionassets.json).

3. **`saveTokensToCsv`**:
   - Maps subjects to their order
   - Saves the output to [``]()

4. **`getAssetsByPolicyId`**:
   - Configures the API request by setting the required headers (including the API key) and query parameters.
   - Sends the request to the API endpoint and handles the response.

In [3]:
type Token = {
    assetId: string
    txCount: int
    mintedOn: string
    policyId: string
    assetName: string
    fingerprint: string
    totalSupply: string
}

type ApiResponse = {
    tokens: Token []
    count: int
    pageNo: int
    limit: int
}

In [4]:
let getApiKey () =
    match Environment.GetEnvironmentVariable("CARDANO_API_KEY") with
    | null -> failwith "API key not found. Set it in your environment variables."
    | key -> key

In [6]:
let saveTokensToCsv (sortedTokens: Token []) (filePath: string) =
    try
        let rows =
            sortedTokens
            |> Array.skip 1 
            |> Array.mapi (fun order token -> sprintf "%s,%d" token.assetId (order + 1))

        let csvContent = StringBuilder()
        csvContent.AppendLine("AssetId,Order") |> ignore
        rows |> Array.iter (fun row -> csvContent.AppendLine(row) |> ignore)

        File.WriteAllText(filePath, csvContent.ToString())
        printfn "CSV file created successfully at %s" filePath
    with ex ->
        printfn "Failed to save CSV: %s" ex.Message

In [7]:
let saveJsonToFile (filePath: string) (jsonResponse: string) =
    try
        
        let data = JsonSerializer.Deserialize<Token []>(jsonResponse)

        let sortedTokens =
            data
            |> Array.toList
            |> List.sortBy (fun token -> token.mintedOn)
            |> List.toArray 

        saveTokensToCsv sortedTokens  "./data/tbc_ranked_by_order.csv"
        
        let options = JsonSerializerOptions(WriteIndented = true, Encoder = System.Text.Encodings.Web.JavaScriptEncoder.UnsafeRelaxedJsonEscaping)
        let jsonOutput = JsonSerializer.Serialize(sortedTokens, options)

        File.WriteAllText(filePath, jsonOutput)
    with ex ->
        printfn "Failed to save JSON at %s. Error: %s" filePath ex.Message

In [8]:
let getAssetsByPolicyId (policyId: string) (filePath: string) =
    async {
        let apiKey = getApiKey()
        let baseUrl = "https://api.cardanoscan.io/api/v1/asset/list/byPolicyId"
        let mutable pageNo = 1
        let mutable allTokens = []

        use client = new HttpClient()
        client.DefaultRequestHeaders.Add("apiKey", apiKey)

        try
            let mutable hasMoreData = true

            while hasMoreData do
                let url = $"{baseUrl}?policyId={policyId}&pageNo={pageNo}"
                let! response = client.GetAsync(url) |> Async.AwaitTask

                if response.IsSuccessStatusCode then
                    let! content = response.Content.ReadAsStringAsync() |> Async.AwaitTask
                    try
                        let data = JsonSerializer.Deserialize<ApiResponse>(content)

                        if data.tokens.Length = 0 then
                            hasMoreData <- false
                        else
                            allTokens <- allTokens @ Array.toList data.tokens
                            pageNo <- pageNo + 1
                    with ex ->
                        printfn "JSON Deserialization Error: %s" ex.Message
                        printfn "Raw Content: %s" content
                        return Error "Failed to deserialize API response." |> ignore
                else
                    let! errorContent = response.Content.ReadAsStringAsync() |> Async.AwaitTask
                    printfn "API Error: %s" errorContent
                    return Error $"Error: {response.StatusCode} - {response.ReasonPhrase}" |> ignore

            let allTokensJson = JsonSerializer.Serialize(allTokens, JsonSerializerOptions(WriteIndented = true))
            saveJsonToFile filePath allTokensJson

            printfn "Successfully saved all tokens to %s" filePath
            return Ok ()
        with ex ->
            printfn "Exception occurred: %s" ex.Message
            return Error $"Exception: {ex.Message}"
    }


### Function Calls

In [9]:
let policyId = "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c"

In [10]:
let filePath = "./data/tbc_roundtwo_minting_order.json"

async {
    let! result = getAssetsByPolicyId policyId filePath
    match result with
    | Ok () -> ()
    | Error errMsg -> printfn "Error: %s" errMsg
}
|> Async.RunSynchronously

CSV file created successfully at ./data/roundtwo_tokens_minting_order.csv
Successfully saved all tokens to ./data/tbc_roundtwo_minting_order.json
